In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
import re

In [3]:
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(3)

In [29]:
df = pd.DataFrame({'번호':[], '분류':[], '제목':[], '만료일':[], '참여인원':[]})
my_list = []
temp_str_list = ['' for x in range(5)]

In [30]:
page_num = 1

while page_num <= 100:
    url = 'https://www1.president.go.kr/petitions/?c=0&only=1&page={}&order=1'.format(page_num)

    # 해당 url 열기
    driver.get(url)
    # 로딩 시간 고려해서 대기 시간 설정
    driver.implicitly_wait(5)

    # xpath 사용해서 tag 검색으로 elements 불러오기
    lis = driver.find_elements(By.CSS_SELECTOR, "ul.petition_list>li>div.bl_wrap")

    
    # 모든 element에 대하여
    for li in lis:
        # tag 사이의 text를 개행 기준으로 나눠서 임시 리스트에 저장
        category = li.find_element(By.CSS_SELECTOR, "div.bl_category.ccategory.cs.wv_category").text
        
        petition_title_elem = li.find_element(By.CSS_SELECTOR, "div.bl_subject>a.cb.relpy_w")
        petition_title = petition_title_elem.text
        petition_code = re.match(r'.+\/([0-9]+)$', petition_title_elem.get_attribute("href")).group(1)
        
        expire_date = li.find_element(By.CSS_SELECTOR, "div.bl_date.light").text
        agree_num = int(li.find_element(By.CSS_SELECTOR, "div.bl_agree.cs").text.replace(",", "").replace("명",""))

        # 임시 리스트를 전체 리스트에 추가
        my_list.append(pd.DataFrame({'번호':[petition_code], '분류':[category], '제목':[petition_title], 
                                     '만료일':[expire_date], '참여인원':[agree_num]}))
            
    page_num = page_num + 1
    time.sleep(10)

In [32]:
# 전체 데이터프레임으로 병합
df = pd.concat(my_list)
# 인덱스 재정렬
df.reset_index(drop=True, inplace=True)

In [33]:
df

,번호,분류,제목,만료일,참여인원
0,602603,행정,제가 국민 2%에 속하는 부자입니까?.,2021-12-29,1639
1,602602,보건복지,영등포구 소재 국공립 어린이집 육아휴직 거부 신고합니다.,2021-12-29,306
2,602601,안전/환경,재활용 불가능한 제품 사용의 규제를 강화해주세요,2021-12-29,157
3,602600,정치개혁,"백신 미접종자, 치료비 자부담 검토 반대",2021-12-29,936
4,602599,안전/환경,우리가 평소에 잘 신경 쓰지 않았던 버려진 담배꽁초 환경오염의 주원인이란 거 알고 ...,2021-12-29,179
...,...,...,...,...,...
566,602037,보건복지,나의 발치한 치아를 이용해 만든 임플란트용 뼈이식재를 진료실에서 제작해 사용할 수 ...,2021-12-01,228
567,602036,보건복지,위드코로나..그리고 백신패스제도..,2021-12-01,2150
568,602035,행정,임대인의 부도덕한 계약요구를 막아주십시요.,2021-12-01,190
569,602034,안전/환경,요양원 치매 환자 등에게 동의 받아 행복도시 소각장 부지를 밀실 불통 행정으로 없애...,2021-12-01,2108


In [34]:
df.to_csv("../data/bluehouse.csv",index = False)